In [39]:

from keras.applications.vgg16 import VGG16


In [40]:
model = VGG16()

In [41]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [42]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
#from keras.applications.imagenet_utils import _obtain_input_shape # this will work for older versions of keras. 2.2.0 or before
#from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import get_source_inputs

from keras_applications.imagenet_utils import _obtain_input_shape

In [43]:
def VGGupdated(input_tensor=None,classes=2):    
   
    img_rows, img_cols = 300, 300   # by default size is 224,224
    img_channels = 3

    img_dim = (img_rows, img_cols, img_channels)
   
    img_input = Input(shape=img_dim)
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    
    # Classification block
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    # Create model.
   
     
    model = Model(inputs = img_input, outputs = x, name='VGGdemo')


    return model

In [44]:
model = VGGupdated(classes = 5)

In [45]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os



dis_types = os.listdir('dataset')
print (dis_types)  #what kinds of rooms are in this dataset

print("Types of disease found: ", len(dis_types))

['beaus_line', 'black_line', 'mees_line', 'onycholysis', 'terrys_nail']
Types of disease found:  5


In [47]:
diseases = []

for item in dis_types:
    all_diseases = os.listdir('dataset' + '/' +item)
    
    for disease in all_diseases:
        diseases.append((item, str('dataset' + '/' +item) + '/' + disease))
        

In [48]:

    
# Build a dataframe        
disease_df = pd.DataFrame(data=diseases, columns=['dis_types', 'image'])
print(disease_df.head())


    dis_types                           image
0  beaus_line  dataset/beaus_line/beau_10.jpg
1  beaus_line  dataset/beaus_line/beau_11.jpg
2  beaus_line  dataset/beaus_line/beau_12.jpg
3  beaus_line  dataset/beaus_line/beau_13.jpg
4  beaus_line  dataset/beaus_line/beau_14.jpg


In [49]:
print("Total number of diseases in the dataset: ", len(disease_df))

disease_count = disease_df['dis_types'].value_counts()

print("disease in each category: ")
print(disease_count)

Total number of diseases in the dataset:  300
disease in each category: 
beaus_line     60
black_line     60
mees_line      60
onycholysis    60
terrys_nail    60
Name: dis_types, dtype: int64


In [50]:
import cv2
path = 'dataset/'


im_size = 300

images = []
labels = []

for i in dis_types:
    data_path = path + str(i)  
    filenames = [i for i in os.listdir(data_path) ]
   
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [51]:
images = np.array(images)

images = images.astype('float32') / 255.0
images.shape 

(300, 300, 300, 3)

In [53]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=disease_df['dis_types'].values
print(y[5:])

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line' 'beaus_line'
 'black_line' 'black_line' 'black_line' 'black_line' 'black_line'
 'black_line' 'black_line' 'black_line' 'black_line' 'black_line'
 'black_line' 'black_line' 'black_line' 'black_line' 'black_line'
 'black_line' 'black_line' 'black_line' 'black_line' 'black_line'
 'black_li

In [56]:
y=y.reshape(-1,1)


In [57]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y).toarray()
print(Y[:-5])

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [58]:
Y

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [59]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(285, 300, 300, 3)
(285, 5)
(15, 300, 300, 3)
(15, 5)


In [ ]:
model.fit(train_x, train_y, epochs = 2, batch_size = 8)  

Epoch 1/2
29/36 [=======================>......] - ETA: 8:37 - loss: 2.3968 - accuracy: 0.1724 

In [ ]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
#print ("Test Accuracy = " + str(preds[1]))

In [ ]:
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow

img_path = 'test_img.jpg'

img = image.load_img(img_path, target_size=(300, 300))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print('Input image shape:', x.shape)

my_image = imread(img_path)
imshow(my_image)

In [ ]:
print(model.predict(x))

In [ ]:
import keras
print(keras.__version__)